In [15]:
!pip install numpy pillow onnx onnxruntime

   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
    --------------------------------------- 0.3/16.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.5 MB 1.4 MB/s eta 0:00:12
   -- ------------------------------------- 1.0/16.5 MB 2.0 MB/s eta 0:00:08
   --- ------------------------------------ 1.6/16.5 MB 2.3 MB/s eta 0:00:07
   ------- -------------------------------- 3.1/16.5 MB 3.3 MB/s eta 0:00:05
   ---------- ----------------------------- 4.2/16.5 MB 3.7 MB/s eta 0:00:04
   ------------- -------------------------- 5.5/16.5 MB 4.1 MB/s eta 0:00:03
   ---------------- ----------------------- 6.8/16.5 MB 4.4 MB/s eta 0:00:03
   ------------------- -------------------- 8.1/16.5 MB 4.6 MB/s eta 0:00:02
   -------------------- ------------------- 8.4/16.5 MB 4.6 MB/s eta 0:00:02
   -------------------- ---

In [16]:
import os
import requests

PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

# Use requests to download the files
def download_file(url, filename):
    if os.path.exists(filename):
        print(f"{filename} already exists. Skipping download.")
        return
    print(f"Downloading {filename}...")
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Downloaded {filename}")

download_file(DATA_URL, "hair_classifier_v1.onnx.data")
download_file(MODEL_URL, "hair_classifier_v1.onnx")

Downloaded hair_classifier_v1.onnx.data
Downloaded hair_classifier_v1.onnx


## Question 1
To be able to use this model, we need to know the name of the input and output nodes.

What's the name of the output:

- output
- sigmoid
- softmax
- prediction

In [17]:
import onnx

model = onnx.load("hair_classifier_v1.onnx")

output_name = model.graph.output[0].name

print(f"The name of the output node is: {output_name}")

The name of the output node is: output


## Question 2: Target size
Let's download and resize this image:

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

- 64x64
- 128x128
- 200x200
- 256x256

In [18]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    # Use Image.Resampling.NEAREST for modern PIL
    img = img.resize(target_size, Image.Resampling.NEAREST) 
    return img

image_url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

In [19]:
target_size = (200, 200)

print(f"The target size for the image is: {target_size}")

The target size for the image is: (200, 200)


## Question 3
Now we need to turn the image into numpy array and pre-process it.

Tip: Check the previous homework. What was the pre-processing we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

- -10.73
- -1.073
- 1.073
- 10.73

In [26]:
import numpy as np

# 1. Download and Resize
img = download_image(image_url)
img = prepare_image(img, target_size)

# 2. Convert to NumPy array (and ensure it's float32 for scaling)
X = np.array(img, dtype='float32')

# 3. Rescale/Normalize (VGG-style normalization)
X /= 255.0  # Scale to 0-1 range
mean = np.array([0.485, 0.456, 0.406]) # Standard ImageNet means
std = np.array([0.229, 0.224, 0.225])   # Standard ImageNet standard deviations

# Apply normalization: (X - mean) / std
X = (X - mean) / std

# Find the value in the first pixel, the R channel
first_pixel_r_value = X[0, 0, 0].round(4) 

print(f"The value in the first pixel (R channel) after pre-processing is: {first_pixel_r_value}")

The value in the first pixel (R channel) after pre-processing is: -1.0733


## Question 4
Now let's apply this model to this image. What's the output of the model?

- 0.09
- 0.49
- 0.69
- 0.89

In [27]:
import numpy as np

X_input = X[np.newaxis, ...] 

X_input = X_input.transpose(0, 3, 1, 2) 

print(f"Final input shape before running model: {X_input.shape}") 

X_input = X_input.astype(np.float32)

sess = onnxruntime.InferenceSession("hair_classifier_v1.onnx")
input_name = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

result = sess.run([output_name], {input_name: X_input})
prediction = result[0][0][0]

print(f"Q4 Answer: The output of the model is: {prediction:.4f}")

Final input shape before running model: (1, 3, 200, 200)
Q4 Answer: The output of the model is: 0.0893
